<br />

<div style="text-align:center;">
<font size = 4.5>課題５ Deferred Acceptance Matchingが提案側の利得を最大にする安定マッチングであることの実証</font>
<font size = 5>金丸博樹</font>
</div>
<br />

<font size = 4>Deferred Acceptance Matchingが提案側の利得を最大にする安定マッチングであることを、考え得る全ての安定マッチングの利得を比較することで示す。</font>

<font size = 4>簡略化のために、男女の１：１マッチングを考え、男女は同数で、選好の長さは相手の人数と同数とする（必ず全員マッチする。）</font>

<font size = 4>まず、男女の数をそれぞれ8人として、ランダムに選好を生成する。</font>

In [1]:
function genprefs(m)
    a=[i for i in 1:m]

    male_prefs=Vector{Vector{Int}}(m)
    female_prefs=Vector{Vector{Int}}(m)

    for i in 1:length(female_prefs)
        female_prefs[i]=shuffle(a)
    end

    for i in 1:length(male_prefs)
        male_prefs[i]=shuffle(a)
    end

    a ,male_prefs, female_prefs
end

genprefs (generic function with 1 method)

In [2]:
m=8
a, male_prefs, female_prefs=genprefs(m)

([1, 2, 3, 4, 5, 6, 7, 8], Array{Int64,1}[[3, 6, 7, 4, 1, 2, 5, 8], [1, 5, 8, 4, 2, 6, 3, 7], [6, 4, 1, 3, 8, 5, 2, 7], [2, 7, 8, 3, 6, 4, 1, 5], [7, 2, 5, 1, 4, 3, 6, 8], [4, 6, 7, 5, 8, 3, 2, 1], [6, 7, 8, 4, 1, 3, 2, 5], [5, 6, 7, 8, 2, 1, 4, 3]], Array{Int64,1}[[8, 7, 6, 1, 4, 3, 5, 2], [6, 7, 3, 2, 8, 5, 1, 4], [7, 8, 6, 5, 4, 3, 1, 2], [8, 7, 5, 4, 1, 3, 6, 2], [5, 7, 4, 8, 3, 6, 1, 2], [3, 1, 7, 6, 8, 2, 5, 4], [6, 1, 5, 3, 4, 7, 8, 2], [5, 7, 4, 8, 3, 2, 1, 6]])

<font size = 4>次に、考え得るマッチングを全て抽出し、安定マッチングのみを抽出する。安定マッチングは、抜け駆けを望むペアが存在しないことである。</font>

In [3]:
using Combinatorics #組み合わせを与えるライブラリを用いる

function stablematches(male_prefs, female_prefs, a, m)
possiblematches=collect(permutations(a)) #男女同数、全員マッチの条件を置いているため、男性のマッチ結果のみで十分

    function stable(possiblematches)
    Boo=Array{Bool}(length(possiblematches))
    Boo[1:end]=true
        for k in 1:length(possiblematches)
            for male in 1:m
                if find(x->(x==possiblematches[k][male]),male_prefs[male])[1]!=1
                    for female in male_prefs[male][1:find(x->(x==possiblematches[k][male]),male_prefs[male])[1]-1]
                        matchedmale=find(x->(x==female),possiblematches[k])[1]
                        if find(x->(x==matchedmale),female_prefs[female])[1]>find(x->(x==male),female_prefs[female])[1]
                            Boo[k]=false
                        #ある男のマッチ相手より上位の女性の中に、その女性のマッチ相手よりその男性を好む人がいる場合、falseを返す
                        end
                    end
                end
            end
        end
    return possiblematches[Boo] #Boolianを用いて、安定でないマッチングを落とす
    end

    stable(possiblematches)
end

stablematches (generic function with 1 method)

In [4]:
st=stablematches(male_prefs, female_prefs, a, m)

7-element Array{Array{Int64,1},1}:
 [3, 1, 6, 2, 7, 4, 8, 5]
 [4, 1, 6, 3, 2, 7, 8, 5]
 [4, 1, 6, 3, 5, 7, 8, 2]
 [4, 2, 6, 3, 5, 7, 8, 1]
 [7, 1, 6, 3, 2, 4, 8, 5]
 [7, 1, 6, 3, 5, 4, 8, 2]
 [7, 2, 6, 3, 5, 4, 8, 1]

<font size = 4>安定マッチングの結果が全て表示された。（男女同数、全員マッチのため、男性のマッチ結果のみを返している）</font>

<font size = 4>ここで、あるマッチングから男性／女性が得る効用を、マッチ相手の自分の中での順位の総和と定義する。（値が小さいほど効用が大きい）</font>

In [5]:
function benefit(stablematches)
    malebenefit=zeros(Int64,length(stablematches))
    femalebenefit=zeros(Int64,length(stablematches))
    totalbenefit=zeros(Int64,length(stablematches))

    for k in 1:length(stablematches)
        for j in 1:m
            malebenefit[k]+=find(x->(x==stablematches[k][j]),male_prefs[j])[1]
            femalebenefit[k]+=find(x->(x==j),female_prefs[stablematches[k][j]])[1]
        end

        totalbenefit[k]=malebenefit[k]+femalebenefit[k]
    end

malebenefit, femalebenefit, totalbenefit
end

benefit (generic function with 1 method)

<font size = 4>実際に男性の効用、女性の効用、両者の和を計算すると以下のようになる。</font>

In [6]:
benefit(st)

([10, 19, 24, 29, 16, 21, 26], [40, 32, 28, 20, 35, 31, 23], [50, 51, 52, 49, 51, 52, 49])

<font size = 4>すなわち、この例だと、最も男性にとって望ましいのは１つ目のマッチ結果で、最も女性にとって望ましいのは４つ目のマッチ結果である。一方、社会全体では４つ目か７つ目が最も望ましい。</font>

<font size = 4>ここで、Deferred Acceptanceによってマッチングをし、男性のマッチ結果を取り出してみる</font>

In [7]:
using MyMatching
match=(my_deferred_acceptance(male_prefs, female_prefs)[1], my_deferred_acceptance(female_prefs, male_prefs)[2])

([3, 1, 6, 2, 7, 4, 8, 5], [4, 2, 6, 3, 5, 7, 8, 1])

<font size = 4>男性が先提案、女性が先提案について、男性のマッチ結果を返している。</font>

<font size = 4>これについて効用を計算してみると、</font>

In [8]:
benefit(match)

([10, 29], [40, 20], [50, 49])

<font size = 4>実際に男性先提案の場合は男性の効用が１０となって安定マッチングの中で最も男性優位に、女性先提案の場合は女性の効用が２０となって安定マッチングの中で最も女性優位になっていることがわかる。</font>